In [1]:
from optimum.onnxruntime import ORTModel
# import re
# from pathlib import Path

# # repo = 'Rostlab/prot_t5_xl_half_uniref50-enc'
# repo = "valentynbez/prot-t5-xl-uniref50-onnx"
# model = ORTModel.from_pretrained(repo)

/nfs/cds-peta/exports/biol_micro_cds_gr_sunagawa/scratch/vbezshapkin/conda-envs/tmvec_slim/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from optimum.onnxruntime import ORTQuantizer, ORTOptimizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig, AutoOptimizationConfig

model = ORTModel.from_pretrained("/nfs/cds-peta/exports/biol_micro_cds_gr_sunagawa/scratch/vbezshapkin/jax-tutorials/onnx/prot_t5_xl_uniref50_og",
                                 file_name="encoder_model.onnx")
optimizer = ORTOptimizer.from_pretrained(model)
oconfig = AutoOptimizationConfig.O3()
optimizer.optimize(
    optimization_config = oconfig,
    save_dir = "onnx/prot_t5_xl_uniref50_onnx_optimized/"
)

The ONNX file encoder_model.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
/nfs/cds-peta/exports/biol_micro_cds_gr_sunagawa/scratch/vbezshapkin/conda-envs/tmvec_slim/lib/python3.11/site-packages/optimum/onnxruntime/configuration.py:770: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(
Optimizing model...
Configuration saved in onnx/prot_t5_xl_uniref50_onnx_optimized/ort_config.json
Optimized model saved at: onnx/prot_t5_xl_uniref50_onnx_optimized (external data format: True; saved all tensor to one file: True)


PosixPath('onnx/prot_t5_xl_uniref50_onnx_optimized')

In [40]:
from datasets import load_dataset

data_files = {
    "train": "ds23_sm.csv"
}
dataset = load_dataset("benchang323/protein-stability-prediction", data_files=data_files)

In [37]:
import re
from transformers import T5Tokenizer
from functools import partial
tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50")

def preprocessing_fn(examples, tokenizer):
    prots = [
                " ".join(list(re.sub(r"[UZOB]", "X", seq)))
                for seq in examples["aa_seq"]
            ]

    return tokenizer.batch_encode_plus(prots, padding=True)

/nfs/cds-peta/exports/biol_micro_cds_gr_sunagawa/scratch/vbezshapkin/conda-envs/tmvec_slim/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [38]:
model = ORTModel.from_pretrained("onnx/prot_t5_xl_uniref50_onnx_optimized")
quantizer = ORTQuantizer.from_pretrained(model)
dataset_id = "benchang323/protein-stability-prediction"
calibration_samples = 1024
calibration_dataset = quantizer.get_calibration_dataset(
    dataset_id,
    preprocess_function=partial(preprocessing_fn, tokenizer=tokenizer),
    num_samples=calibration_samples
)


# qconfig = AutoQuantizationConfig.avx2(is_static=False, per_channel=False)

# quantizer.quantize(
#     quantization_config = qconfig,
#     save_dir = "onnx/prot_t5_xl_uniref50_onnx_quantized/"
# )

The ONNX file encoder_model_optimized.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
/nfs/cds-peta/exports/biol_micro_cds_gr_sunagawa/scratch/vbezshapkin/conda-envs/tmvec_slim/lib/python3.11/site-packages/datasets/load.py:2547: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
Generating train split: 776298 examples [00:01, 694221.01 examples/s]


DatasetGenerationCastError: An error occurred while generating the dataset

All the data files must have the same columns, but at some point there are 1 new columns ({'deltaG_bin'})

This happened while the csv dataset builder was generating data using

hf://datasets/benchang323/protein-stability-prediction/ds23_sm_resampled.csv (at revision 27d61b6f9744887dafc07a0e6286cd2fc5bcc012)

Please either edit the data files to have matching columns, or separate them into different configurations (see docs at https://hf.co/docs/hub/datasets-manual-configuration#multiple-configurations)

In [12]:


proteins = [
    "MTEYKLVVVGAGGVGKSALTIQLIQNHFVDEY",
    "MTEYKLVVWWGKKKKGKSALTIQLIQNHFVDEY",
]
proteins =
inp = tokenizer.batch_encode_plus(proteins, return_tensors="pt", padding=True)

/nfs/cds-peta/exports/biol_micro_cds_gr_sunagawa/scratch/vbezshapkin/conda-envs/tmvec_slim/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
onnx_input = {
    model.get_inputs()[0].name: inp["input_ids"].numpy(),
    model.get_inputs()[1].name: inp["attention_mask"].numpy(),
}

output = model.run(None, onnx_input)

In [16]:
output

[array([[[ 0.07404899, -0.20570587, -0.2021014 , ...,  0.07341778,
           0.18258561,  0.10436805],
         [ 0.26948896,  0.24683222, -0.10092371, ...,  0.04219685,
           0.13717978, -0.1400615 ],
         [ 0.20434596,  0.0652505 ,  0.34895548, ..., -0.14778548,
          -0.09826204,  0.14222495],
         ...,
         [-0.03607703, -0.1470879 , -0.17805701, ..., -0.20021366,
          -0.09398003,  0.20085084],
         [-0.04174141, -0.08354487,  0.01946584, ..., -0.10311703,
          -0.01141954, -0.06496002],
         [ 0.09838589, -0.18436186, -0.00223857, ...,  0.01989791,
          -0.12956178, -0.01691233]],
 
        [[ 0.02059202, -0.29090127,  0.06930815, ...,  0.30442306,
           0.03168807, -0.06918357],
         [ 0.15663372, -0.11146327, -0.10829254, ...,  0.1446364 ,
           0.2825544 , -0.01001046],
         [ 0.0575058 , -0.07439127,  0.17446327, ...,  0.20658618,
           0.17073876, -0.0530398 ],
         ...,
         [-0.02416883,  0.0488689